In [1]:
import json
import math
class Problem:
    def __init__(self, input):
        self.input = input
        self.numTasks = len(input)
    def cost(self, ans):
        totalTime = 0
        for task, agent in enumerate(ans):
            totalTime += self.input[task][agent]
        return totalTime


if __name__ == '__main__':
    # row: task
    # col: worker
    input = [[10, 20, 23, 4],
            [15, 13, 6, 25],
            [ 2, 22, 35, 34],
            [12, 3, 14, 17]
            ]

    # ans: ans[i] mean task i for which agent
    ans = [3, 2, 0, 1] # ⽤演算法得出的答案
    solver = Problem(input)
    print('Assignment:', ans) 
    print('Cost:', solver.cost(ans)) 

Assignment: [3, 2, 0, 1]
Cost: 15


In [ ]:
    # import random

    # width = 11
    # input = [[0] * width] * width
    # for i in range(width):
    #     for j in range(width):
    #         input[i][j] = random.random()
        
    input =[[0.43045255, 0.78681387, 0.07514408, 0.72583933, 0.52916145, 0.87483212, 0.34701621],
            [0.68704291, 0.45392742, 0.46862110, 0.67669006, 0.23817468, 0.87520581, 0.67311418],
            [0.38505150, 0.05974168, 0.11388629, 0.28978058, 0.66089373, 0.92592403, 0.70718757],
            [0.24975701, 0.16937649, 0.42003672, 0.88231235, 0.74635725, 0.59854858, 0.88631100], 
            [0.64895582, 0.58909596, 0.99772334, 0.85522575, 0.33916707, 0.72873479, 0.26826203],
            [0.47939038, 0.88484586, 0.05122520, 0.83527995, 0.37219939, 0.20375257, 0.50482283],
            [0.58926554, 0.45176739, 0.25217475, 0.83548120, 0.41687026, 0.00293049, 0.23939052]]

In [4]:
import json

# Read json files
with open("input.json", 'r') as f:
    input_datas = json.load(f)
for key, input in input_datas.items():
    print(key)
    print(input)

0
[[0.26300727684204517, 0.48513471953446996, 0.8491417036699047], [0.7518785807425733, 0.029752222747783996, 0.5887209536993653], [0.7761974553100254, 0.19546118308946114, 0.6158427400193519]]
1
[[0.029027546749207156, 0.9596392488940668, 0.054290564347959425], [0.8792382327744873, 0.4842657230586963, 0.5985550149648665], [0.2892086218859313, 0.3626276202039942, 0.6700932067358675]]
2
[[0.6522038778448653, 0.28423090494141323, 0.05590092696332172], [0.2836377068594478, 0.09781821643160182, 0.6822000693953171], [0.14111826833715846, 0.8575015422476077, 0.7231015975001829]]
3
[[0.647014824270405, 0.5743436653420422, 0.34113078456058654], [0.4654658024389251, 0.9898657107200758, 0.03872021770708478], [0.268902984084759, 0.05321547844433949, 0.1327202215246065]]
4
[[0.739168054388306, 0.5112623311338964, 0.33032336722590006], [0.9690603029875725, 0.5912395531612223, 0.9805000116454766], [0.6678436037935193, 0.03758394798617892, 0.17314364493622258]]


In [62]:
import random

def get_range(num_genes:int):
    start_idx = random.randint(0, num_genes)
    end_idx = random.randint(0, num_genes)
    if start_idx > end_idx:
        tmp = start_idx
        start_idx = end_idx
        end_idx = tmp
    return start_idx, end_idx
class Mutation(object):
    def __init__(self, population:list):
        self.population = population
        self.num_genes = len(self.population[0])

    def __call__(self, p1:int, c1:int):
        raise NotImplementedError
    
class Inversion_mutation(Mutation):
    def __init__(self, population: list):
        super().__init__(population)
    def __call__(self, p1: int, c1: int):
        start_idx, end_idx = get_range(self.num_genes) 
        for i in range(self.num_genes):
            if(i >= start_idx and i < end_idx):
                reverse_idx = (end_idx - 1) - (i - start_idx)
                self.population[c1][i] = self.population[p1][reverse_idx]
            else:
                self.population[c1][i] = self.population[p1][i]
class Swap_mutation(Mutation):
    def __init__(self, population: list):
        super().__init__(population)
    def __call__(self, p1: int, c1: int):
        num_swap = 1 #int(self.num_genes * 0.1)

        # Copy parent
        for i in range(self.num_genes):
            self.population[c1][i] = self.population[p1][i]
        for _ in range(num_swap):
            from_idx = random.randint(0, self.num_genes - 1)
            to_idx = random.randint(0, self.num_genes - 1)
            if from_idx == to_idx:
                continue
            tmp = self.population[c1][to_idx]
            self.population[c1][to_idx] = self.population[c1][from_idx]
            self.population[c1][from_idx] = tmp

class Scramble_mutation(Mutation):
    def __init__(self, population: list):
        super().__init__(population)
    def __call__(self, p1: int, c1: int):
        start_idx, end_idx = get_range(self.num_genes) 
        # Copy parent
        for i in range(self.num_genes):
            self.population[c1][i] = self.population[p1][i]
        random.shuffle(self.population[c1][start_idx:end_idx])
class Selection(object):
    def __call__(self, fitness_list:list, num_select:int):
        raise NotImplementedError

class Wheel_selection(Selection):
    """Roulette Wheel Selection
    """
    def __call__(self, fitness_list:list, num_select:int):
        indices = [i for i in range(len(fitness_list))]
        
        selected_indices = random.choices(indices, weights=fitness_list, k=num_select)
        return selected_indices
    
class Deterministic_selection(Selection):
    def __call__(self, fitness_list:list, num_select:int):
        num_items = len(fitness_list)
        sorted_indices = sorted(range(num_items), key=lambda k : fitness_list[k])
        sorted_indices.reverse()

        return sorted_indices[:num_select]


In [70]:
import random
import sys

class Problem:
    def __init__(self, input):
        self.input = input
        self.numTasks = len(input)
    def cost(self, ans):
        totalTime = 0
        for task, agent in enumerate(ans):
            totalTime += self.input[task][agent]
        return totalTime

MAX_INT = sys.maxsize
def assign_array(target, source):
    for i in range(len(target)):
        target[i] = source[i]
class GA(object):
    def __init__(self, problem:Problem, 
                        pop_size : int,
                        iter_times : int,
                        cross_ratio : float,
                        mut_ratio : float,
                        selection_method : Selection,
                        mut_method : Mutation,
                        ):
        self.problem = problem
        self.num_genes = self.problem.numTasks
        self.pop_size = pop_size # population size
        self.cross_size = int(self.pop_size * cross_ratio) # crossover size
        
        # Cross size should be even
        if self.cross_size % 2 != 0:
            self.cross_size += 1
        self.mut_size = int(self.pop_size * mut_ratio) # mutation size
        self.total_size = self.pop_size + self.cross_size + self.mut_size
        self.iter_times = iter_times

        self.index_template = [i for i in range(self.total_size)]
        # init population
        self.population = [self.gen_random_indices(self.num_genes) for _ in range(self.total_size)]
        self.seleted_parents = [[0] * self.num_genes for _ in range(self.pop_size)]
        self.objetive = [0] * self.total_size
        self.fitness = [0] * self.total_size
        self.best_fitness = -MAX_INT
        self.best_obj = MAX_INT
        self.best_obj_ans = None
        self.best_ans = None

        # Methods
        self.mut_method = mut_method(self.population)
        self.selection_method = selection_method()
    def gen_random_indices(self, length):
        indices = self.index_template[:length].copy()
        random.shuffle(indices)
        return indices

    def do_cross_over(self):
        indices = self.gen_random_indices(self.pop_size)
        for i in range(0, self.cross_size, 2):
            p1_idx = indices[i] # parent1
            p2_idx = indices[i + 1] # parent2
            c1_idx = self.pop_size + i
            c2_idx = self.pop_size + i + 1
            self.partially_mapped_crossover(p1_idx, p2_idx, c1_idx, c2_idx)

    def partially_mapped_crossover(self, p1, p2, c1, c2):
        """
        Args:
            p1: the index of parent1
            p2: the index of parent1
            c1: the index of offspring1
            c2: the index of offspring1
        """
        start_idx, end_idx = get_range(self.num_genes) 
        mapping1 = [-1] * self.num_genes
        mapping2 = [-1] * self.num_genes
        for i in range(start_idx, end_idx):
            # swap
            v1 = self.population[p1][i]
            v2 = self.population[p2][i]
            self.population[c1][i] = v2
            self.population[c2][i] = v1
            mapping1[v2] = v1
            mapping2[v1] = v2
        
        for i in range(self.num_genes):
            if i >= start_idx and i < end_idx:
                continue
            v1 = self.population[p1][i]
            v2 = self.population[p2][i]

            while mapping1[v1] != -1:
                v1 = mapping1[v1]
            while mapping2[v2] != -1:
                v2 = mapping2[v2]
            self.population[c1][i] = v1
            self.population[c2][i] = v2
    
    def do_mutation(self):
        # Parents and their children mutates
        indices = self.gen_random_indices(self.pop_size + self.cross_size)
        
        mut_idx = self.pop_size + self.cross_size
        for i in range(self.mut_size):
            self.mut_method(indices[i], mut_idx)
            mut_idx += 1


    def evaluate_fitness(self):
        min_obj = MAX_INT
        max_obj = -MAX_INT
        # Compute objective values
        for i, ans in enumerate(self.population):
            cost = self.problem.cost(ans)
            if cost < min_obj:
                min_obj = cost
            if cost > max_obj:
                max_obj = cost  
            self.objetive[i] = cost
            if cost < self.best_obj:
                self.best_obj = cost
                self.best_obj_ans = ans.copy()

        range_obj = max_obj - min_obj
        # Compute fitness
        least_fitness_factor = 0.3
        for i, obj in enumerate(self.objetive):
            fitness = max(least_fitness_factor * range_obj, pow(10,-5)) + (max_obj - obj)
            self.fitness[i] = fitness
            # Update best fitness and ans
            if fitness > self.best_fitness:
                self.best_fitness = fitness
                self.best_ans = self.population[i].copy()
    
    def do_selection(self):
        num_select = self.pop_size
        seleted_indices = self.selection_method(self.fitness, num_select)

        for i, seleted_idx in enumerate(seleted_indices):
            assign_array(self.seleted_parents[i], self.population[seleted_idx])
        for i in range(num_select):
            assign_array(self.population[i], self.seleted_parents[i])
    def solve(self):
        for i in range(self.iter_times):
            self.do_cross_over()
            self.do_mutation()
            self.evaluate_fitness()
            self.do_selection()

In [1]:
from GA import GA, Problem

input =[[0.43045255, 0.78681387, 0.07514408, 0.72583933, 0.52916145, 0.87483212, 0.34701621],
        [0.68704291, 0.45392742, 0.46862110, 0.67669006, 0.23817468, 0.87520581, 0.67311418],
        [0.38505150, 0.05974168, 0.11388629, 0.28978058, 0.66089373, 0.92592403, 0.70718757],
        [0.24975701, 0.16937649, 0.42003672, 0.88231235, 0.74635725, 0.59854858, 0.88631100],
        [0.64895582, 0.58909596, 0.99772334, 0.85522575, 0.33916707, 0.72873479, 0.26826203],
        [0.47939038, 0.88484586, 0.05122520, 0.83527995, 0.37219939, 0.20375257, 0.50482283],
        [0.58926554, 0.45176739, 0.25217475, 0.83548120, 0.41687026, 0.00293049, 0.23939052]]

problem = Problem(input)
solver = GA(problem)
# ans = solver.best_ans
solver.solve()
obj_ans = solver.best_obj_ans
print('Assignment:', obj_ans)
print('Cost:', problem.cost(obj_ans)) 

Assignment: None


TypeError: 'NoneType' object is not iterable

In [134]:
input = [[10, 20, 23, 4],
        [15, 13, 6, 25],
        [ 2, 22, 35, 34],
        [12, 3, 14, 17]
        ]
input =[[0.43045255, 0.78681387, 0.07514408, 0.72583933, 0.52916145, 0.87483212, 0.34701621],
        [0.68704291, 0.45392742, 0.46862110, 0.67669006, 0.23817468, 0.87520581, 0.67311418],
        [0.38505150, 0.05974168, 0.11388629, 0.28978058, 0.66089373, 0.92592403, 0.70718757],
        [0.24975701, 0.16937649, 0.42003672, 0.88231235, 0.74635725, 0.59854858, 0.88631100],
        [0.64895582, 0.58909596, 0.99772334, 0.85522575, 0.33916707, 0.72873479, 0.26826203],
        [0.47939038, 0.88484586, 0.05122520, 0.83527995, 0.37219939, 0.20375257, 0.50482283],
        [0.58926554, 0.45176739, 0.25217475, 0.83548120, 0.41687026, 0.00293049, 0.23939052]]


problem = Problem(input)



GA_solver = GA(problem=problem, 
            pop_size=100, 
            iter_times=300,
            cross_ratio=0.2,
            mut_ratio=0.1,
            selection_method=Wheel_selection,
            mut_method=Inversion_mutation,
        )

GA_solver.solve()
ans = GA_solver.best_ans
obj_ans = GA_solver.best_obj_ans
print('Assignment:', ans)
print('Cost:', problem.cost(ans)) 

print('Assignment:', obj_ans)
print('Cost:', problem.cost(obj_ans)) 

Assignment: [6, 4, 3, 0, 1, 2, 5]
Cost: 1.7679801299999998
Assignment: [0, 4, 3, 1, 6, 2, 5]
Cost: 1.45020202


In [115]:

import random
from BF import Exhaustive
random.seed(12345)
width = 11
input = [[0] * width] * width
for i in range(width):
    for j in range(width):
        input[i][j] = random.random()
problem = Problem(input)
Ex_solver = Exhaustive(problem)
Ex_ans = Ex_solver.get_ans()
print('Ex Ans=', Ex_ans)
print('Ex Cost:', problem.cost(Ex_ans)) 

Ex Ans= [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 4]
Ex Cost: 4.254500948720415


In [116]:
GA_solver = GA(problem=problem, 
            pop_size=100, 
            iter_times=300,
            cross_ratio=0.2,
            mut_ratio=0.1,
            selection_method=Deterministic_selection,
            mut_method=Inversion_mutation,
        )

GA_solver.solve()
ans = GA_solver.best_ans
obj_ans = GA_solver.best_obj_ans
print('Assignment:', ans)
print('Cost:', problem.cost(ans)) 

print('Assignment:', obj_ans)
print('Cost:', problem.cost(obj_ans)) 

Assignment: [0, 8, 1, 9, 10, 3, 2, 6, 5, 7, 4]
Cost: 4.254500948720415
Assignment: [0, 8, 1, 9, 10, 3, 2, 6, 5, 7, 4]
Cost: 4.254500948720415


In [20]:
solver.population

[[1, 6, 3, 0, 4, 2, 5],
 [1, 2, 3, 4, 6, 5, 0],
 [1, 2, 6, 0, 4, 3, 5],
 [3, 4, 2, 5, 6, 1, 0],
 [4, 6, 2, 5, 0, 1, 3],
 [0, 4, 2, 3, 6, 1, 5],
 [4, 0, 2, 1, 5, 6, 3],
 [3, 4, 2, 5, 6, 1, 0],
 [6, 0, 1, 3, 5, 2, 4],
 [1, 2, 3, 4, 6, 5, 0],
 [3, 6, 2, 4, 5, 1, 0],
 [1, 2, 3, 4, 6, 5, 0],
 [0, 1, 3, 4, 2, 6, 5],
 [3, 0, 2, 5, 6, 1, 4],
 [1, 2, 3, 4, 6, 5, 0],
 [1, 6, 3, 0, 4, 2, 5],
 [6, 0, 2, 5, 4, 1, 3],
 [1, 0, 6, 3, 5, 2, 4],
 [4, 2, 0, 5, 6, 1, 3],
 [1, 2, 3, 5, 6, 4, 0],
 [3, 6, 2, 4, 5, 1, 0],
 [0, 6, 1, 2, 4, 3, 5],
 [3, 0, 2, 5, 6, 1, 4],
 [5, 0, 3, 6, 4, 1, 2],
 [3, 5, 4, 2, 6, 1, 0],
 [6, 5, 1, 0, 4, 2, 3],
 [1, 2, 6, 0, 4, 3, 5],
 [0, 6, 1, 2, 4, 3, 5],
 [4, 6, 2, 5, 0, 1, 3],
 [4, 0, 2, 1, 5, 6, 3],
 [4, 3, 1, 0, 5, 2, 6],
 [1, 6, 3, 0, 4, 2, 5],
 [6, 0, 2, 5, 4, 1, 3],
 [4, 1, 2, 0, 6, 5, 3],
 [4, 2, 6, 1, 5, 3, 0],
 [4, 1, 0, 2, 6, 5, 3],
 [1, 2, 6, 0, 4, 3, 5],
 [6, 5, 1, 0, 4, 2, 3],
 [4, 6, 2, 5, 0, 1, 3],
 [5, 0, 3, 6, 4, 1, 2],
 [2, 3, 4, 6, 0, 5, 1],
 [2, 4, 5, 0, 3,